Interface creation employs Zur and McGill matching superlattices algorithm


In [ ]:
from utils import install_packages

await install_packages(verbose=False)
from definitions import get_materials, set_materials, to_pymatgen, from_pymatgen, plot_strain_vs_atoms
from params import (
    zsl_widgets,
    interface_widgets,
    update_params,
    display_form,
    get_zsl_generator_params,
    get_interface_params,
    get_plot_settings,
)

display_form(zsl_widgets, interface_widgets, update_params)

In [ ]:
get_materials()

In [ ]:
from pymatgen_coherent_interface_builder import CoherentInterfaceBuilder, ZSLGenerator
from operator import itemgetter

pymatgen_materials = [to_pymatgen(item) for item in data]
for material in pymatgen_materials:
    print(material)


def create_interfaces(settings):
    # Create Interface Builder class
    zsl = ZSLGenerator(
        max_area_ratio_tol=settings["MAX_AREA_TOL"],
        max_area=settings["MAX_AREA"],
        max_length_tol=settings["MAX_LENGTH_TOL"],
        max_angle_tol=settings["MAX_ANGLE_TOL"],
    )

    cib = CoherentInterfaceBuilder(
        substrate_structure=pymatgen_materials[settings["SUBSTRATE_INDEX"]],
        film_structure=pymatgen_materials[settings["LAYER_INDEX"]],
        substrate_miller=settings["SUBSTRATE_MILLER"],
        film_miller=settings["LAYER_MILLER"],
        zslgen=zsl,
        strain_tol=settings["STRAIN_TOL"],
    )

    # Run the Interface Building process
    cib._find_terminations()
    matches = cib.zsl_matches
    terminations = cib.terminations

    # Create interfaces
    interfaces = []
    for termination in terminations:
        interfaces = list(
            cib.get_interfaces(
                termination,
                gap=settings["DISPLACEMENT_Z"],
                film_thickness=settings["LAYER_THICKNESS"],
                substrate_thickness=settings["SUBSTRATE_THICKNESS"],
                in_layers=True,
            )
        )

    print(f"Found {len(matches)} interfaces")
    print(f"Found {len(terminations)} terminations:", terminations)

    strain_modes = {
        "VON_MISES": "von_mises_strain",
        "STRAIN": "strain",
        "MEAN": "mean_abs_strain",
    }
    strain_mode = strain_modes["MEAN"]
    interfaces_list = list(interfaces)

    # Sort interfaces by ascending strain and then by ascending number of atoms
    sorted_interfaces = sorted(
        interfaces_list,
        key=lambda x: (itemgetter(strain_mode)(x), x["interface"].num_sites),
    )

    print("Interface with lowest strain (index 0):")
    print("    strain:", sorted_interfaces[0][strain_mode] * 100, "%")
    print("    number of atoms:", sorted_interfaces[0]["interface"].num_sites)
    PS = get_plot_settings()
    # plot stran vs number of atoms via matplotlib
    plot_strain_vs_atoms(strain_mode, sorted_interfaces, settings=PS)
    return sorted_interfaces


ZP = get_zsl_generator_params()
IP = get_interface_params()
interfaces = create_interfaces(settings={**ZP, **IP})

In [ ]:
m3_esse = from_pymatgen(interfaces[0]["interface"])
set_materials([m3_esse])